In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
import torchvision
from torchvision import datasets, models, transforms

In [3]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches, patheffects
import pandas as pd
import json
import cv2
from PIL import ImageDraw, ImageFont
from collections import namedtuple, OrderedDict

In [4]:
from session import *
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageClassifierData as ImageClassifierData
from Transforms.ImageTransforms import *

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
torch.cuda.is_available(), torch.cuda.get_device_name(0), torch.backends.cudnn.enabled

(True, 'GeForce GTX 960M', True)

In [7]:
DATA_PATH = Path('C:/fastai/courses/dl2/data/road_damage_dataset')

In [8]:
MULTICLASS_CSV_PATH = DATA_PATH/'mc.csv'
MULTIBB_CSV_PATH = DATA_PATH/'mb.csv'

In [9]:
im_size = 224

In [10]:
train_tfms = TransformList([
        RandomScale(im_size, 1.2),
        RandomCrop(im_size),
        RandomLighting(0.05, 0.05),
        RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_tfms = TransformList([
        Scale(im_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [11]:
batch_size = 8
data = ImageClassifierData.from_csv(DATA_PATH, MULTICLASS_CSV_PATH, batch_size, train_tfms, val_tfms)
num_classes = 8

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
    nn.BatchNorm1d(num_features=num_ftrs),
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, num_classes)
)

In [14]:
criterion = nn.BCEWithLogitsLoss()
optim_fn = optim.Adam

In [15]:
sess = Session(model_ft, criterion, optim_fn, [*[1e-3] * 9, 1e-2])

In [16]:
sess.load('FullTrain')

In [35]:
x,y,*_ = next(iter(data['valid']))

In [36]:
meta = _[0]
meta['file'][0]

'C:\\fastai\\courses\\dl2\\data\\road_damage_dataset\\Sumida\\JPEGImages\\train_Sumida_00508.jpg'

In [26]:
nn.Sigmoid()(sess.forward(x))

Variable containing:
 0.2095  0.2418  0.0407  0.0238  0.3559  0.0773  0.0108  0.3572
 0.1891  0.4023  0.1011  0.1344  0.6164  0.0363  0.0019  0.0673
 0.2577  0.3949  0.1201  0.1116  0.4763  0.0467  0.0066  0.1739
 0.3530  0.3678  0.0727  0.0463  0.2051  0.0399  0.0131  0.5572
 0.2711  0.6150  0.0464  0.1131  0.0703  0.0143  0.0237  0.4205
 0.1561  0.5147  0.0260  0.0554  0.1477  0.0161  0.0118  0.3997
 0.1480  0.1386  0.0360  0.0224  0.1454  0.0267  0.7559  0.1279
 0.3909  0.3096  0.0717  0.0629  0.2014  0.0350  0.0136  0.6256
[torch.cuda.FloatTensor of size 8x8 (GPU 0)]

In [27]:
y


    0     0     0     0     1     0     0     0
    0     1     0     0     0     0     0     0
    0     0     0     0     1     1     0     0
    0     1     0     0     1     0     0     0
    0     1     0     0     0     0     0     1
    0     1     0     0     0     0     0     1
    1     0     0     0     0     0     1     0
    1     0     0     0     0     0     0     1
[torch.FloatTensor of size 8x8]

In [37]:
sess.model

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNo